In [21]:
import json
import numpy as np
import random
from transformers import BertTokenizer
import pandas as pd

In [8]:
#Make sure these are divisible by 4
with open("baseline_prompt.txt", "r") as file:
    prompt = file.read()

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
encoded_input = tokenizer(prompt, return_tensors="pt")

PROMPT_OVERHEAD = encoded_input['input_ids'].shape[1]
TRAIN_DATA_POINTS = 8000
TEST_DATA_POINTS = 2000
TOTAL_EXAMPLES = TRAIN_DATA_POINTS + TEST_DATA_POINTS
MAX_RATING = 5
MAX_TOKENS = 512 - PROMPT_OVERHEAD

assert TRAIN_DATA_POINTS % 4 == 0
assert TEST_DATA_POINTS % 4 == 0

In [35]:
file = "Books.jsonl"
train_example_cnts = np.zeros(MAX_RATING + 1)
test_example_cnts = np.zeros(MAX_RATING + 1)

train_data = []

test_data = []

with open(file, 'r') as fp:
    for line in fp:
            
        train_data_done = np.sum(train_example_cnts) >= TRAIN_DATA_POINTS
        test_data_done = np.sum(test_example_cnts) >= TEST_DATA_POINTS
        
        if train_data_done and test_data_done:
            break
            
        review = json.loads(line.strip())
        rating = int(review["rating"])
        text = review["text"]
        encoded_input = tokenizer(text, return_tensors="pt")
        
        if (encoded_input['input_ids'].shape[1] > MAX_TOKENS):
            continue    
        
        if (review["rating"] != 3):
            #randomnly pick train or test
            dataset_choice = random.choice([0, 1])
            if (not train_data_done) and dataset_choice == 0 and (train_example_cnts[rating] != (TRAIN_DATA_POINTS / 4)):
                train_example_cnts[rating] += 1
                sentiment = "positive" if rating > 3 else "negative"
                train_data.append([text, sentiment])
            elif (not test_data_done) and (test_example_cnts[rating] != (TEST_DATA_POINTS / 4)):
                test_example_cnts[rating] += 1
                sentiment = "positive" if rating > 3 else "negative"
                test_data.append([text, sentiment])

assert len(train_data) == TRAIN_DATA_POINTS
assert len(test_data)  == TEST_DATA_POINTS

AssertionError: 

In [42]:
train_df = pd.DataFrame(train_data, columns=['Review', 'Sentiment'])
train_df = train_df.sample(frac=1).reset_index(drop=True)

test_df = pd.DataFrame(test_data, columns=['Review', 'Sentiment'])
test_df = test_df.sample(frac=1).reset_index(drop=True)

In [44]:
train_df.to_csv('data/train_data.csv', index=False)
test_df.to_csv('data/test_data.csv', index=False)